# Obtaining the block_ranking

In [1]:
import pandas as pd
import pickle
import networkx as nx
import queue
import itertools

In [2]:
df = pd.read_csv('wiki-topcats-page-names.txt',sep='\n', header=-1, names=["Name"])
df.head()

,Name
0,0 Chiasmal syndrome
1,1 Kleroterion
2,2 Pinakion
3,3 LyndonHochschildSerre spectral sequence
4,4 Zariski's main theorem


In [3]:
#df.index=df.apply(lambda x:x['Name'].split()[0], axis =1) #probably useless
df["Name"]=df.apply(lambda x: " ".join(x["Name"].split()[1:]),axis=1)
df.head()

,Name
0,Chiasmal syndrome
1,Kleroterion
2,Pinakion
3,LyndonHochschildSerre spectral sequence
4,Zariski's main theorem


In [4]:
n=len(df)
n

1791489

In [5]:
category = pd.read_csv("wiki-topcats-categories.txt", header = None, sep = "/n", names=["nodes"],engine="python")

In [6]:
category.head()

,nodes
0,Category:Buprestoidea; 301 302 303 304 305 306...
1,Category:People_from_Worcester; 1056 1057 1058...
2,Category:Skin_conditions_resulting_from_physic...
3,Category:Visual_kei_bands; 1297 1300 1311 1312...
4,Category:Japanese_rock_music_groups; 1297 1300...


In [7]:
category["Name"]=category.apply(lambda x: x["nodes"].split(";")[0].split(":")[1], axis=1)

In [8]:
category.head()

,nodes,Name
0,Category:Buprestoidea; 301 302 303 304 305 306...,Buprestoidea
1,Category:People_from_Worcester; 1056 1057 1058...,People_from_Worcester
2,Category:Skin_conditions_resulting_from_physic...,Skin_conditions_resulting_from_physical_factors
3,Category:Visual_kei_bands; 1297 1300 1311 1312...,Visual_kei_bands
4,Category:Japanese_rock_music_groups; 1297 1300...,Japanese_rock_music_groups


In [9]:
category["nodes"]= category.apply(lambda x: list(map(int,x["nodes"].split(";")[1].split())),axis=1)

In [10]:
#just change the order of the columns
category= category[["Name","nodes"]]

In [11]:
cat = category.loc[category.apply(lambda x: len(x["nodes"]), axis = 1) > 3500]

In [12]:
c = len(cat)
c

35

In [13]:
cat.head()

,Name,nodes
868,English_footballers,"[22860, 28411, 28961, 28979, 29264, 29573, 295..."
869,The_Football_League_players,"[14003, 23536, 27109, 27348, 27459, 27989, 280..."
876,Association_football_forwards,"[26876, 26877, 26879, 26887, 26892, 26904, 269..."
898,Association_football_goalkeepers,"[26900, 26909, 26917, 26960, 26966, 26984, 270..."
900,Association_football_midfielders,"[14003, 15291, 23536, 26880, 26882, 26885, 268..."


In [14]:
lengths=[]
for i,val in cat.iterrows():
    lengths.append(len(val["nodes"]))
lengths
#so I take the third last wich has no many nodes

[9237,
 9467,
 6959,
 3997,
 8270,
 6668,
 418223,
 3760,
 6154,
 6580,
 6546,
 5913,
 7851,
 3813,
 34721,
 7729,
 13704,
 5701,
 4853,
 3501,
 4551,
 22699,
 15302,
 3697,
 4888,
 3542,
 11661,
 13938,
 8401,
 12174,
 7237,
 6767,
 3588,
 4040,
 4517]

In [15]:
with open("Graph", "rb") as f:
    graph = pickle.load(f)

In [16]:
#input category 
C0 = "American_military_personnel_of_World_War_II"
nodes_C0 = list(cat.loc[cat["Name"]==C0]["nodes"])[0]

nodes_C0

[1231,
 12401,
 13319,
 14341,
 14807,
 15902,
 15913,
 15915,
 15917,
 15918,
 16319,
 16406,
 16408,
 16410,
 16420,
 16472,
 16541,
 16558,
 16580,
 16692,
 16712,
 17054,
 17058,
 17065,
 17088,
 17100,
 17115,
 17118,
 17146,
 17165,
 17167,
 17171,
 17175,
 17176,
 17210,
 17424,
 17799,
 17809,
 17815,
 17817,
 17911,
 17914,
 18067,
 18669,
 18673,
 18735,
 18830,
 18858,
 18859,
 18869,
 18880,
 18881,
 18882,
 18883,
 18886,
 18887,
 18888,
 18894,
 18896,
 18949,
 18974,
 18975,
 18977,
 18978,
 18979,
 18981,
 18982,
 18984,
 18985,
 18988,
 18989,
 18990,
 18993,
 19002,
 19004,
 19010,
 19015,
 19016,
 19017,
 19019,
 19022,
 19023,
 19024,
 19027,
 19032,
 19036,
 19039,
 19043,
 19045,
 19046,
 19048,
 19050,
 19051,
 19052,
 19053,
 19057,
 19059,
 19063,
 19066,
 19068,
 19075,
 19082,
 19083,
 19084,
 19086,
 19089,
 19094,
 19099,
 19110,
 19112,
 19114,
 19121,
 19122,
 19125,
 19128,
 19129,
 19132,
 19138,
 19191,
 19201,
 19202,
 19204,
 19208,
 19209,
 19217,
 

In [17]:
#using a queue
def breadth_first_search(G,a):
    visited=[False]*n
    visited[a]=True
    q = queue.Queue()
    q.put(a)
    d={}
    d[a]=0
    while not q.empty():
        v=q.get()
        for i in list(G.neighbors(v)):
            if not visited[i]:
                q.put(i)
                visited[i]=True
                d[i] = d[v]+1
    return d

In [18]:
bfs1 =nx.single_source_shortest_path_length(G=graph,source=52)
our_bfs =breadth_first_search(graph, 52)
bfs1 == our_bfs

True

Our BFS is correct

Now we need to compute the dictionary of the distances (using BFS) with source every node belonging to the category $C_0$. 

We would create a dictionary of this format:

- The key are all the nodes of the graph
- For each key the value is a list, where each element is the lenght of the shortest path with one of the nodes of the category $C_0$

But we would encounter a memory error (the previous dictionary has about 400k keys, each one with value a list of about 3.5k elements).

Then we store as values (instead of a list) a nested dictionary, with keys the distances and values the FREQUENCIES with which each distance appear.

In [53]:
# !!! This cell would take 2 hours, DO NOT RUN IT !!!
# Instead if you want you can upload the "final_dict" (2 cells later)

final_dict=dict((i,{}) for i in graph.nodes)
for i in nodes_C0:
    if i in graph:
        d = breadth_first_search(graph,i)
        for j in d:
            final_dict[j][d[j]] = final_dict[j].get(d[j],0) +1

In [54]:
with open("final_dict","wb") as f:
    pickle.dump(final_dict,f)

Now we can load it:

In [19]:
with open("final_dict","rb") as f:
    final_dict = pickle.load(f)

In [51]:
final_dict

{52: {4: 3033, 3: 395, 5: 244, 6: 25, 7: 2, 8: 1},
 62: {2: 2167, 3: 1265, 4: 249, 5: 16, 6: 3},
 64: {3: 2189, 4: 1339, 5: 148, 2: 10, 6: 13, 7: 1},
 66: {2: 2136, 3: 1283, 4: 260, 5: 18, 6: 3},
 74: {2: 2098, 3: 1284, 4: 297, 5: 18, 6: 3},
 95: {4: 2917, 5: 531, 3: 205, 6: 43, 7: 3, 8: 1},
 103: {4: 2248, 5: 1284, 6: 154, 7: 12, 8: 2},
 104: {3: 2103, 4: 1317, 5: 260, 6: 17, 7: 3},
 105: {3: 2350, 4: 1097, 2: 162, 5: 84, 6: 6, 7: 1},
 107: {3: 2248, 4: 1284, 5: 154, 6: 12, 7: 2},
 108: {3: 2323, 4: 1192, 5: 165, 6: 12, 2: 6, 7: 2},
 112: {4: 2248, 5: 1284, 6: 154, 7: 12, 8: 2},
 113: {2: 2098, 3: 1285, 4: 296, 5: 18, 6: 3},
 122: {3: 2350, 4: 1097, 2: 162, 5: 84, 6: 6, 7: 1},
 126: {2: 2098, 3: 1284, 4: 297, 5: 18, 6: 3},
 133: {2: 2098, 3: 1284, 4: 297, 5: 18, 6: 3},
 134: {3: 2647, 4: 886, 2: 60, 5: 98, 6: 8, 7: 1},
 137: {3: 2430, 4: 1134, 5: 110, 6: 11, 2: 14, 7: 1},
 153: {3: 2361, 4: 1177, 5: 150, 6: 11, 7: 1},
 154: {2: 2098, 3: 1303, 4: 278, 5: 18, 6: 3},
 155: {2: 2111, 3: 1

Let's see how it looks like, just for two nodes:

In [52]:
{52: final_dict[52], 62:final_dict[62]}

{52: {4: 3033, 3: 395, 5: 244, 6: 25, 7: 2, 8: 1},
 62: {2: 2167, 3: 1265, 4: 249, 5: 16, 6: 3}}

Now for each category $C_i$, we can easily obtain the __ShortestPath($C_0$,$C_i$)__ just taking all the dictionaries for every node belonging to $C_i$, and merging them.

Once obtained the set  ShortestPath($C_0$,$C_i$), we just need to take the median of it.

We could compute the median using `statistics.median`, but in the code of that function it is uesd sorted(list) instead of list.sort().

Now the problem is that for the category Living_people which has 400k nodes, the list shortest_path is very large, and if we use sorted() we are creating a copy of this, that could create a memory error. So we have to create a little function to compute the median by ourself:

In [20]:
def median(l):
    l.sort()
    middle = len(l)//2
    return l[middle]

In [21]:
medians={}
list_of_names = cat.loc[cat["Name"]!=C0]["Name"]
for Ci in list_of_names:
    nodes_Ci = list(cat.loc[cat["Name"]==Ci]["nodes"])[0]
    shortest_path=[]
    for node in nodes_Ci:
        try:
            d = final_dict[node]
            for lenght in d:
                shortest_path.extend(d[lenght]*[lenght])
        except:
            continue
    medians[Ci]= median(shortest_path)

In [22]:
medians

{'English_footballers': 4,
 'The_Football_League_players': 4,
 'Association_football_forwards': 4,
 'Association_football_goalkeepers': 4,
 'Association_football_midfielders': 4,
 'Association_football_defenders': 4,
 'Living_people': 4,
 'Year_of_birth_unknown': 4,
 'Harvard_University_alumni': 3,
 'Major_League_Baseball_pitchers': 4,
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 4,
 'Indian_films': 4,
 'Year_of_death_missing': 4,
 'English_cricketers': 4,
 'Year_of_birth_missing_(living_people)': 4,
 'Rivers_of_Romania': 3,
 'Main_Belt_asteroids': 4,
 'Asteroids_named_for_people': 4,
 'English-language_albums': 3,
 'English_television_actors': 4,
 'British_films': 4,
 'English-language_films': 3,
 'American_films': 3,
 'Fellows_of_the_Royal_Society': 4,
 'People_from_New_York_City': 3,
 'American_Jews': 3,
 'American_television_actors': 3,
 'American_film_actors': 3,
 'Debut_albums': 4,
 'Black-and-white_films': 3,
 'Year_of_birth_missing': 4,
 'Place_of_bir

In [23]:
with open("medians","wb") as f:
    pickle.dump(medians,f)

In [24]:
with open("medians","rb") as f:
    medians = pickle.load(f)

Just sort the dictionary by values:

In [35]:
block_ranking = [k for k, v in sorted(medians.items(), key=lambda x: x[1])]

In [38]:
block_ranking

['Harvard_University_alumni',
 'Rivers_of_Romania',
 'English-language_albums',
 'English-language_films',
 'American_films',
 'People_from_New_York_City',
 'American_Jews',
 'American_television_actors',
 'American_film_actors',
 'Black-and-white_films',
 'Place_of_birth_missing_(living_people)',
 'Article_Feedback_Pilot',
 'English_footballers',
 'The_Football_League_players',
 'Association_football_forwards',
 'Association_football_goalkeepers',
 'Association_football_midfielders',
 'Association_football_defenders',
 'Living_people',
 'Year_of_birth_unknown',
 'Major_League_Baseball_pitchers',
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
 'Indian_films',
 'Year_of_death_missing',
 'English_cricketers',
 'Year_of_birth_missing_(living_people)',
 'Main_Belt_asteroids',
 'Asteroids_named_for_people',
 'English_television_actors',
 'British_films',
 'Fellows_of_the_Royal_Society',
 'Debut_albums',
 'Year_of_birth_missing',
 'Windows_games']

# Sort the nodes in each category